In [2]:
import sys
import os
sys.path.insert(0, os.path.abspath(os.path.join('..', '..')))
from src import *
from itertools import product

import numpy as np
import pandas as pd
import scipy.optimize as opt

from matplotlib import pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

In [3]:
moments = np.array([-2, -1, 2, 3, 4])
ps = np.array([0.85, 0.9, 0.95, 1.0, 1.05, 1.1, 1.15])
system_sizes = np.array([100, 200, 400, 800, 1600, 3200, 6400])
realizations = 10

multifractal_data = np.load("../../data/multifractal_data.npy")

In [4]:
# this is an (Nx5) matrix where each row has the following structure
# row 1: IPR_avg_1 | IPR_err_1 | L_1 | p_1 | lamb_1
# row 2: IPR_avg_2 | IPR_err_2 | L_2 | p_2 | lamb_2
# and so on...

ipr_data = np.load("../../data/AVG_IPR_joined_data.npy")
ipr_df = pd.DataFrame(ipr_data, columns = ['average', 'error', 'L', 'p', 'lambda'])
ipr_ps = np.sort(pd.unique(ipr_df['p']))
ipr_Ls = np.array(np.sort(pd.unique(ipr_df['L'])), dtype = int)


In [5]:
def f(x, a, c2): return c2*(x**a)

powers = []
gradients = []
for p in ipr_ps:
    curve_data = ipr_df[ipr_df['p'] == p]
    res = opt.curve_fit(
        f, curve_data['L'], 1/curve_data['average'], 
        bounds=([0, -np.inf], [np.inf, np.inf])
    )
    powers.append(res[0][0])
    gradients.append(res[0][1])

def f2(x, a): return x**a

powers2 = []
for p in ipr_ps:
    curve_data = ipr_df[ipr_df['p'] == p]
    res = opt.curve_fit(
        f2, curve_data['L'], 1/curve_data['average'], 
        bounds=([0], [np.inf])
    )
    powers2.append(res[0][0])

In [50]:
%matplotlib qt

plot_moment = [0, 1, 1, 1, 0]
plot_p = [0, 0, 1, 1, 1, 1, 0]

# Plotting parameters
scaling = 1.0
title_font_size = 10*scaling
plt.rcParams['xtick.major.pad'] = '0'#'-1.5'
plt.rcParams['ytick.major.pad'] = '0'#'-1.5'
plt.rcParams.update({
    "font.family": "Times New Roman",
    "font.size": 10*scaling,
    "text.usetex": True,
})

# Figure setup
pt = 0.0138889
fig, ax = plt.subplots(
    figsize=(246*pt*scaling, 250*0.4*pt*scaling),
    gridspec_kw={
        'left': 0.12, 'right': 0.99, 'top': 0.965, 'bottom': 0.165, 'hspace': 0.4,
    }
)

# IPR data
for L in ipr_Ls:
    if L != 1000:
        ax.plot(
            ipr_ps, 
            1/np.array(ipr_df[ipr_df['L'] == L].sort_values('p')['average']),
            ls = '--'
        )
        ax.scatter(
            ipr_ps,
            1/np.array(ipr_df[ipr_df['L'] == L].sort_values('p')['average']),
            marker='o',
            label=r'{}'.format(L),
            s=5
        )
ax.set_xlabel(r"$p$", labelpad=-10*scaling)
ax.set_ylabel(r"$\mathrm{PR}$", labelpad=-1)
ax.set_xticks((0, 2, 4, 6, 8, 10))
ax.tick_params(axis='x', pad=2)
ax.set_yscale('log')
#ax[0].legend(
#    loc=7, 
#    bbox_to_anchor=(0.0, 0.0, 1.22, 1.0), 
#    borderpad=0.1, 
#    handletextpad=0.0,
#    handlelength=1.0
#)

ax.annotate("L", xy=(0.5, 10.0), xytext=(1.9, 400),
            arrowprops=dict(arrowstyle="<|-", facecolor="black", lw=1.5),
            bbox=dict(pad=-1, facecolor="none", edgecolor="none"))

# IPR inset
inset_ax = inset_axes(ax, width="50%", height="55%", loc=4,
                   bbox_to_anchor=(.0, .32, 0.95, 0.98), bbox_transform=ax.transAxes)
inset_ax.set_xlabel(r"$p$", labelpad=-9*scaling)
#inset_ax.set_title(r"$\mathrm{PR}(p)=p^a/c$", fontsize=title_font_size, pad=-10)
inset_ax.set_xticks((0, 4))

color = (0.05, 0.4, 0.6)
inset_ax.plot(ipr_ps[ipr_ps <= 4.0], np.abs(powers)[ipr_ps <= 4.0], color=color)
inset_ax.set_ylabel(r"$a$", color=color, labelpad=-9*scaling)
inset_ax.tick_params(axis='y', labelcolor=color)
inset_ax.tick_params(axis='both', pad=2)

#color = (1.0, 0.5, 0.2)
#inset_ax_twin = inset_ax.twinx()
#inset_ax_twin.plot(ipr_ps, 1/np.abs(gradients), color=color)
#inset_ax_twin.set_ylabel(r"$c$", color=color, labelpad=-9*scaling)
#inset_ax_twin.tick_params(axis='y', labelcolor=color)
#inset_ax_twin.set_yticks((0.0, 2.0))
#inset_ax_twin.tick_params(axis='y', pad=2)

plt.show()


In [51]:
plt.savefig("../../paper_figs/participation_ratio.eps")


In [69]:
%matplotlib qt

plot_moment = [0, 1, 1, 1, 0]
plot_p = [0, 0, 1, 1, 1, 1, 0]

# Plotting parameters
scaling = 1.0
title_font_size = 10*scaling
plt.rcParams['xtick.major.pad'] = '0'  # '-1.5'
plt.rcParams['ytick.major.pad'] = '0'  # '-1.5'
plt.rcParams.update({
    "font.family": "Times New Roman",
    "font.size": 10*scaling,
    "text.usetex": True,
})

# Figure setup
pt = 0.0138889
fig2, ax = plt.subplots(
    nrows=3, figsize=(246*pt*scaling, 250*0.7*pt*scaling),
    gridspec_kw={
        'left': 0.135, 'right': 0.99, 'top': 0.955, 'bottom': 0.105, 'hspace': 0.5,
        'height_ratios': (0.33, 0.33, 0.33)
    }
)

y_labels = [r"$D_{-2}$", r"$D_{-1}$", r"$D_{2}$", r"$D_{3}$", r"$D_{4}$"]
labels = [r"$\mathrm{(a)}$", r"$\mathrm{(b)}$", r"$\mathrm{(c)}$"]
# Multifractality data
for (k, q) in enumerate(moments):
    if plot_moment[k]:
        for (i, p) in enumerate(ps):
            if plot_p[i]:
                ax[k-1].scatter(
                    system_sizes, 
                    multifractal_data[i, :, k],
                    linewidth=2.0, label="{}".format(p),
                    s=5
                )
                ax[k-1].plot(
                    system_sizes, 
                    multifractal_data[i, :, k], 
                    linewidth=0.5,
                    ls='--'
                )
        ax[k-1].set_xlabel(r"$L$", labelpad=-9*scaling)
        ax[k-1].set_ylabel(y_labels[k], labelpad=-1)
        ax[k-1].set_xscale('log')
        #ax[k].set_title(r"$q={}$".format(q), fontsize=title_font_size, pad=-10)
        ax[k-1].tick_params(axis='y', pad=1)
        ax[k-1].text(-0.15, 0.95, labels[k-1], transform = ax[k-1].transAxes)

ax[0].annotate("p", xy=(150.0, 1.09), xytext=(300.0, 1.18),
            arrowprops=dict(arrowstyle="<|-", facecolor="black", lw=1.5),
            bbox=dict(pad=-1, facecolor="none", edgecolor="none"))
ax[1].annotate("p", xy=(150.0, 0.87), xytext=(300.0, 0.735),
            arrowprops=dict(arrowstyle="<|-", facecolor="black", lw=1.5),
            bbox=dict(pad=-1, facecolor="none", edgecolor="none"))
ax[2].annotate("p", xy=(150.0, 0.83), xytext=(300.0, 0.683),
            arrowprops=dict(arrowstyle="<|-", facecolor="black", lw=1.5),
            bbox=dict(pad=-1, facecolor="none", edgecolor="none"))
#ax[2].legend(
#    loc=7, 
#    bbox_to_anchor=(0.0, 0.0, 1.215, 1.0),
#    borderpad=0.1,
#    handletextpad=0.0,
#    handlelength=1.0
#)

plt.show()


In [70]:
plt.savefig("../../paper_figs/fractal_dimensions.eps")

In [71]:
%matplotlib qt

# Plotting parameters
scaling = 1.0
title_font_size = 10*scaling
plt.rcParams['xtick.major.pad'] = '0'  # '-1.5'
plt.rcParams['ytick.major.pad'] = '0'  # '-1.5'
plt.rcParams.update({
    "font.family": "Times New Roman",
    "font.size": 10*scaling,
    "text.usetex": True,
})

# Figure setup
pt = 0.0138889
fig3, ax = plt.subplots(
    nrows = 2, figsize=(246*pt*scaling, 200*pt*scaling),
    gridspec_kw={
        'left': 0.13, 'right': 0.93, 'top': 0.975, 'bottom': 0.085, 'hspace': 0.25,
        'height_ratios': (0.5, 0.5)
    }
)

ax[0].set_xlabel(r"$p$", labelpad=-9*scaling)
#ax[0].set_title(r"$\mathrm{PR}(p)=p^a/c$", fontsize=title_font_size, pad=-10)
ax[0].set_xticks((0, 1, 3, 4))

color = (0.05, 0.4, 0.6)
ax[0].plot(ipr_ps[ipr_ps <= 4.0], np.abs(powers)[ipr_ps <= 4.0], color=color)
ax[0].set_ylabel(r"$a$", color=color, labelpad=-9*scaling)
ax[0].tick_params(axis='y', labelcolor=color)
ax[0].tick_params(axis='both', pad=2)
ax[0].set_yticks((0.0, 1.0))
ax[0].text(-0.15, 0.95, r"$\mathrm{(a)}$", transform = ax[0].transAxes)

color = (1.0, 0.5, 0.2)
ax_twin = ax[0].twinx()
ax_twin.plot(ipr_ps[ipr_ps <= 4.0], 1/np.abs(gradients)[ipr_ps <= 4.0], color=color)
ax_twin.set_ylabel(r"$c$", color=color, labelpad=-9*scaling)
ax_twin.tick_params(axis='y', labelcolor=color)
ax_twin.set_yticks((0.0, 2.0))
ax_twin.tick_params(axis='y', pad=2)

ax[1].set_xlabel(r"$p$", labelpad=-9*scaling)
#ax[1].set_title(r"$\mathrm{PR}(p)=p^a/c$", fontsize=title_font_size, pad=-10)
ax[1].set_xticks((0, 1, 3, 4))

color = (0.05, 0.4, 0.6)
ax[1].plot(ipr_ps[ipr_ps <= 4.0], np.abs(powers2)[ipr_ps <= 4.0], color=color)
ax[1].set_ylabel(r"$a$", color=color, labelpad=-9*scaling)
ax[1].tick_params(axis='y', labelcolor=color)
ax[1].tick_params(axis='both', pad=2)
ax[1].set_yticks((0.0, 1.0))
ax[1].text(-0.15, 0.95, r"$\mathrm{(b)}$", transform = ax[1].transAxes)

Text(-0.15, 0.95, '$\\mathrm{(b)}$')

In [72]:
plt.savefig("../../paper_figs/participation_ratio_fit.eps")